<h1>Objetivo</h1>
<h5><b>Avaliação:</b></h5><p>Para avaliar a performance da previsão do timestamp datahora, será usada a métrica RMSE. Para avaliar a performance da previsão de latitude e longitude, será usada a métrica Mean Haversine Distance</p>

In [ ]:
#Manipulação de arquivos
import os
import zipfile
from glob import glob
import json
import gc
import warnings

#Manipulação e processamento de dados
import pandas as pd
import numpy as np
import cudf
import cupy as cp

#Machine Learning
from cuml.cluster import DBSCAN
from sklearn.cluster import DBSCAN as SklearnDBSCAN
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import torch
import joblib
import optuna
import xgboost as xgb
from cuml.ensemble import RandomForestRegressor as cuRF
from catboost import CatBoostRegressor

#Barra de progresso
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

#Visualização
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates

#Perfil de memória
from memory_profiler import memory_usage


warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
#Caminhos das pastas
base_dir = os.getcwd()
dados_dir = os.path.join(base_dir, 'dados')
intermediarios_dir = os.path.join(dados_dir, 'intermediarios')
processados_dir = os.path.join(dados_dir, 'processados')

#Criar diretórios se não existirem
os.makedirs(intermediarios_dir, exist_ok=True)
os.makedirs(processados_dir, exist_ok=True)

#Lista global para armazenar os DataFrames processados
final_df_list = []

#Dicionário para armazenar o sentido atual de cada ônibus
sentidos_atual = {}

<h1>EDA</h1>
<p>Fazendo algumas análises estatísticas e plotagens de dados</p>

In [ ]:
class EDA:
    def __init__(self, file_path):
        #Inicializa a classe com o caminho do arquivo de dados processados.
        self.file_path = file_path  

    #Carrega os dados do arquivo Parquet usando pandas ou cudf.
    def load_data(self, use_cudf=False):
        try:
            if use_cudf:
                return cudf.read_parquet(self.file_path) 
            else:
                return pd.read_parquet(self.file_path) 
        except Exception as e:
            print(f"Erro ao carregar os dados: {e}")
            return None

    #Realiza uma inspeção inicial dos dados, visualizando as primeiras linhas e um resumo estatístico.
    def initial_inspection(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            print(df.describe()) 
            del df  
            gc.collect() 

    #Analisa as estatísticas dos dados, incluindo a distribuição das velocidades e a contagem de registros por linha e sentido.
    def analyze_statistics(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            #Plota a distribuição das velocidades
            df['velocidade'].hist(bins=50)  # Plota um histograma
            plt.title('Distribuição de Velocidades')
            plt.xlabel('Velocidade')
            plt.ylabel('Frequência')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo x
            plt.show()

            #Conta registros por linha e sentido e exibe os resultados
            #print(df.groupby(['linha', 'sentido']).size().to_frame('count').reset_index())
            del df 
            gc.collect()

    #Analisa padrões temporais dos dados, incluindo o volume de dados por hora e padrões de movimentação ao longo do tempo.
    def analyze_temporal_patterns(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            #Extrai a hora do timestamp
            #df['datahora_converted'] = pd.to_datetime(df['datahora'])
            df['hora'] = df['datahora_converted'].dt.hour

            # Filtra as horas entre 8 e 22
            df = df[(df['hora'] >= 8) & (df['hora'] < 22)]

            #Conta registros por hora e plota o volume de dados por hora
            hourly_counts = df.groupby('hora').size().to_frame('count').reset_index()
            hourly_counts.plot(x='hora', y='count', kind='bar')
            plt.title('Volume de Dados por Hora')
            plt.xlabel('Hora do Dia')
            plt.ylabel('Contagem')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.show()

            #Conta registros por dia ao longo do tempo e plota padrões de movimentação
            df['data'] = df['datahora_converted'].dt.date  # Extrai apenas a data
            daily_counts = df.groupby('data').size().to_frame('count').reset_index()
            daily_counts.plot(x='data', y='count', kind='line')
            plt.title('Padrões de Movimentação ao Longo do Tempo')
            plt.xlabel('Data')
            plt.ylabel('Contagem')
            plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Formata o eixo x para datas
            plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
            plt.xticks(rotation=45)
            plt.show()

            del df  
            gc.collect()  

    #Analisa padrões geográficos dos dados, plotando um mapa de calor dos trajetos dos ônibus.
    def analyze_geographical_patterns(self, linhas=None):
        try:
            df_cudf = self.load_data(use_cudf=True)
            if df_cudf is not None:
                df_cudf['linha'] = df_cudf['linha'].astype(str)  # Garante que a coluna 'linha' seja string

                if linhas:
                    linhas = list(map(str, linhas))
                    for linha in linhas:
                        if linha not in df_cudf['linha'].to_pandas().tolist():
                            raise ValueError(f"Linha {linha} não encontrada nos dados.")

                    df_cudf = df_cudf[df_cudf['linha'].isin(linhas)]

                num_linhas = len(linhas) if linhas else len(df_cudf['linha'].unique())
                sample_fraction = min(1.0 / num_linhas, 0.01)  # Ajusta a fração de amostra de acordo com o número de linhas
                max_points = min(5000 * num_linhas, 50000)  # Ajusta o número máximo de pontos de acordo com o número de linhas

                #Define o centro do mapa baseado na média de latitude e longitude usando cudf
                map_center = [float(df_cudf['latitude'].mean()), float(df_cudf['longitude'].mean())]
                map_osm = folium.Map(location=map_center, zoom_start=12)  # Cria um mapa centrado na localização média

                #Mostra os dados antes de converter para pandas para evitar problemas de memória
                if len(df_cudf) > max_points:
                    df_sample = df_cudf.sample(frac=sample_fraction)
                    if len(df_sample) > max_points:
                        df_sample = df_sample.sample(n=max_points)
                else:
                    df_sample = df_cudf

                df_pandas = df_sample.to_pandas()

                #Dados para o mapa de calor
                heat_data = [[row['latitude'], row['longitude']] for index, row in df_pandas[['latitude', 'longitude']].iterrows()]
                HeatMap(heat_data).add_to(map_osm)  # Adiciona o mapa de calor ao mapa

                title = 'Mapa de Calor de Todas as Linhas'
                if linhas:
                    title = f'Mapa de Calor das Linhas: {", ".join(map(str, linhas))}'
                map_osm.get_root().html.add_child(folium.Element(f"<h1>{title}</h1>"))

                map_osm.save('bus_routes.html')  # Salva o mapa como um arquivo HTML
                print(f"Mapa de calor salvo como 'bus_routes.html' com {len(df_pandas)} pontos de amostra")
                
                del df_cudf  
                del df_pandas  
                gc.collect()
            else:
                print("Erro ao carregar os dados.")
        except ValueError as ve:
            print(ve)
        except Exception as e:
            print(f"Erro ao gerar o mapa de calor: {e}")

    #Plota os trajetos dos ônibus, locais de garagem e pontos finais com diferentes cores.
    def plot_routes_and_endpoints(self, linhas=None):
        try:
            df_cudf = self.load_data(use_cudf=True)
            if df_cudf is not None:
                df_cudf['linha'] = df_cudf['linha'].astype(str)  # Garante que a coluna 'linha' seja string

                if linhas:
                    linhas = list(map(str, linhas))
                    for linha in linhas:
                        if linha not in df_cudf['linha'].to_pandas().tolist():
                            raise ValueError(f"Linha {linha} não encontrada nos dados.")

                    df_cudf = df_cudf[df_cudf['linha'].isin(linhas)]

                num_linhas = len(linhas) if linhas else len(df_cudf['linha'].unique())
                sample_fraction = min(1.0 / num_linhas, 0.01)  # Ajusta a fração de amostra de acordo com o número de linhas
                max_points = min(5000 * num_linhas, 50000)  # Ajusta o número máximo de pontos de acordo com o número de linhas

                #Define o centro do mapa baseado na média de latitude e longitude usando cudf
                map_center = [float(df_cudf['latitude'].mean()), float(df_cudf['longitude'].mean())]
                map_osm = folium.Map(location=map_center, zoom_start=12)  # Cria um mapa centrado na localização média

                #Mostra os dados antes de converter para pandas para evitar problemas de memória
                if len(df_cudf) > max_points:
                    df_sample = df_cudf.sample(frac=sample_fraction)
                    if len(df_sample) > max_points:
                        df_sample = df_sample.sample(n=max_points)
                else:
                    df_sample = df_cudf

                df_pandas = df_sample.to_pandas()

                #Plota os trajetos normais
                normal_routes = df_pandas[~df_pandas['garagem'] & ~df_pandas['ponto_final']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in normal_routes[['latitude', 'longitude']].iterrows()],
                        name='Trajetos Normais', gradient={0.4: 'blue', 1: 'lightblue'}).add_to(map_osm)

                #Plota os locais de garagem
                garage_routes = df_pandas[df_pandas['garagem']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in garage_routes[['latitude', 'longitude']].iterrows()],
                        name='Locais de Garagem', gradient={0.4: 'red', 1: 'darkred'}).add_to(map_osm)

                #Plota os pontos finais
                end_points = df_pandas[df_pandas['ponto_final']]
                HeatMap([[row['latitude'], row['longitude']] for index, row in end_points[['latitude', 'longitude']].iterrows()],
                        name='Pontos Finais', gradient={0.4: 'yellow', 1: 'gold'}).add_to(map_osm)

                title = 'Mapa de Trajetos, Locais de Garagem e Pontos Finais de Todas as Linhas'
                if linhas:
                    title = f'Mapa de Trajetos, Locais de Garagem e Pontos Finais das Linhas: {", ".join(map(str, linhas))}'
                map_osm.get_root().html.add_child(folium.Element(f"<h1>{title}</h1>"))

                map_osm.save('bus_routes_endpoints.html')  # Salva o mapa como um arquivo HTML
                print(f"Mapa de trajetos e pontos salvos como 'bus_routes_endpoints.html' com {len(df_pandas)} pontos de amostra")

                del df_cudf  #Libera a memória
                del df_pandas  #Libera a memória
                gc.collect()  #Coleta o lixo
            else:
                print("Erro ao carregar os dados.")
        except ValueError as ve:
            print(ve)
        except Exception as e:
            print(f"Erro ao gerar o mapa de trajetos e pontos: {e}")

    #Analisa a distribuição dos pontos finais e pontos de garagem.
    def analyze_endpoints_garage(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            #Plota pontos finais
            df[df['ponto_final'] == True][['latitude', 'longitude']].plot(kind='scatter', x='longitude', y='latitude')
            plt.title('Distribuição de Pontos Finais')
            plt.show()

            #Plota pontos de garagem
            df[df['garagem'] == True][['latitude', 'longitude']].plot(kind='scatter', x='longitude', y='latitude')
            plt.title('Distribuição de Pontos de Garagem')
            plt.show()

            del df  #Libera a memória
            gc.collect()  #Coleta o lixo

    #Analisa os trajetos dos ônibus para cada linha.
    def analyze_routes(self):
        df = self.load_data(use_cudf=False)
        if df is not None:
            linhas = df['linha'].unique()  # Obtém todas as linhas únicas
            num_linhas = len(linhas)
            num_cols = 3
            num_rows = (num_linhas + num_cols - 1) // num_cols  # Calcula o número de linhas para os subplots
            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
            axes = axes.flatten()

            for idx, linha in enumerate(linhas):
                trajeto = df[df['linha'] == linha]  # Filtra os dados pela linha
                ax = axes[idx]
                ax.plot(trajeto['longitude'], trajeto['latitude'], label=f'Linha {linha}')  # Plota o trajeto
                ax.set_title(f'Trajeto da Linha {linha}')
                ax.set_xlabel('Longitude')
                ax.set_ylabel('Latitude')
                ax.legend()
                ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo y
                ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'))  # Formata o eixo x

            # Remove subplots vazios
            for idx in range(len(linhas), len(axes)):
                fig.delaxes(axes[idx])

            plt.tight_layout()
            plt.show()

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def check_data_quality(self):
        """
        Verifica a qualidade dos dados, incluindo valores nulos, dados inconsistentes e outliers.
        """
        df = self.load_data(use_cudf=False)
        if df is not None:
            # Verifica valores nulos em cada coluna
            print("Verificando valores nulos...")
            print(df.isnull().sum())

            # Verifica valores não numéricos na coluna 'velocidade'
            print("Verificando dados inconsistentes...")
            problematic_values = df[~df['velocidade'].astype(str).str.isnumeric()]
            if not problematic_values.empty:
                print(f"Valores problemáticos em 'velocidade':\n{problematic_values[['velocidade', 'velocidade']].head()}")

            # Verifica outliers de velocidade
            print("Verificando outliers de velocidade [Velocidade acima de 150km/h]...")
            outliers = df[(df['velocidade'] < 0) | (df['velocidade'] > 150)]
            print(f"Outliers:\n{outliers}")

            del df  # Libera a memória
            gc.collect()  # Coleta o lixo

    def feature_engineering(self, df):
        """
        Cria novas features a partir das informações temporais.
        """
        print("Criando novas features...")
        #df['datahora_converted'] = pd.to_datetime(df['datahora'])
        df['dia_da_semana'] = df['datahora_converted'].dt.dayofweek  # Adiciona a feature 'dia_da_semana'
        df['hora'] = df['datahora_converted'].dt.hour  # Adiciona a feature 'hora'
        print("Novas features criadas: 'dia_da_semana', 'hora'")
        return df


    def run_all(self):
        """
        Executa todos os métodos de análise exploratória de dados em sequência.
        """
        self.initial_inspection()
        self.analyze_statistics()
        self.analyze_temporal_patterns()
        self.analyze_geographical_patterns()
        self.plot_routes_and_endpoints()
        self.analyze_endpoints_garage()
        self.analyze_routes()
        self.check_data_quality()


In [ ]:
eda = EDA(file_path=os.path.join(processados_dir, 'processed_data.parquet'))

In [ ]:
#Realizar inspeção inicial
eda.initial_inspection()

In [ ]:
#Analisar estatísticas
eda.analyze_statistics()

In [ ]:
#Analisar padrões temporais
eda.analyze_temporal_patterns()

In [ ]:
#Analisar padrões geográficos
eda.analyze_geographical_patterns(linhas=["324", "3"])

In [ ]:
#Plotar trajetos e pontos
eda.plot_routes_and_endpoints(linhas=["324", "3"])

In [ ]:
#Analisar pontos finais e pontos de garagem
eda.analyze_endpoints_garage()

In [ ]:
#Analisar trajetos dos ônibus
eda.analyze_routes()


In [ ]:
#Verificar qualidade dos dados
eda.check_data_quality()

<h1>Treinando 2 horas de dados consecutivas para prever a próxima hora</h1>

<h3>Extraindo os dados dos arquivos .zip e concatenando os arquivos .json:</h3>
<p>Nessa parte do código a ideia é extrair os arquivos .json dos arquivos .zip. Cada arquivo .zip contém 2 tipos de dados: dados históricos e dados de teste. Portanto, ao final, todos os arquivos de dados históricos foram concatenados em 1 arquivo e todos os arquivos de teste foram concatenados em outro arquivo.</p>
<h3>2 tipos de arquivos de teste:</h3>
<p>Existem 2 arquivos de teste: Um para testar previsão de datahora e outro para testar a previsão de latitude e longitude. Portanto, 2 arquivos de teste foram criados</p>
<h3>Filtragem Inicial:</h3>
<p>Para atender as demandas do enunciado do professor Zimbrão, algumas filtragens iniciais foram feitas:
<li>apenas linhas específicas que serão analizadas</li>
<li>considerei apenas os limites de velocidade entre 0 e 250</li>
<li>considerei apenas os limites de latitude e longitude possíveis: entre -90 e 90 e entre -180 e 180</li>
<li>eu adicionei os 2 últimos dígitos de cada arquivo .json em uma nova coluna para ambos os datasets de treino e teste e chamei essa nova coluna de hour_from_file. O objetivo é usar essa coluna como referência na hora de treinar as 2 últimas horas para prever a hora seguinte</li>

</p>

In [ ]:
class FinalDataProcessor:
    def __init__(self, base_dir):
        #Inicializa diretórios e cria pastas necessárias
        self.base_dir = os.path.join(base_dir, 'dados_finais')
        self.intermediarios_teste_dir = os.path.join(self.base_dir, 'intermediarios_teste')
        self.processados_teste_dir = os.path.join(self.base_dir, 'processados_teste')
        self.chunk_size = 100000

        os.makedirs(self.intermediarios_teste_dir, exist_ok=True)
        os.makedirs(self.processados_teste_dir, exist_ok=True)

    #Função para extrair arquivos ZIP
    def extract_zip_files(self, zip_files):
        #Itera sobre cada arquivo ZIP
        for zip_file in tqdm(zip_files, desc="Extracting zip files"):
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall(self.intermediarios_teste_dir)

    #Função para processar arquivos JSON
    def process_json(self, json_path, df_list):
        #Abre e carrega o arquivo JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        #Converte para DataFrame cuDF
        df = cudf.DataFrame.from_pandas(pd.DataFrame(data))

        #Adiciona coluna com a hora extraída do nome do arquivo
        hour_from_file = int(json_path.split('_')[-1].split('.')[0])
        df['hour_from_file'] = hour_from_file

        #Adiciona DataFrame à lista
        df_list.append(df)

    #Função para processar e concatenar arquivos
    def process_and_concat_files(self, prefix, output_parquet, is_test=False):
        all_files = []

        #Caminha pelos diretórios e adiciona arquivos JSON que começam com o prefixo especificado
        for root, _, files in os.walk(self.intermediarios_teste_dir):
            for file in files:
                if file.startswith(prefix) and file.endswith('.json'):
                    all_files.append(os.path.join(root, file))

        final_df_list = []

        #Itera sobre cada arquivo JSON e processa
        for json_file in tqdm(all_files, desc=f"Processing {prefix} files"):
            self.process_json(json_file, final_df_list)

        #Concatena DataFrames se a lista não estiver vazia
        if final_df_list:
            final_df = cudf.concat(final_df_list, ignore_index=True)
            final_df.to_parquet(output_parquet)

            if is_test:
                self.split_test_data(final_df)
            else:
                final_df.to_csv(output_parquet.replace('.parquet', '.csv'), index=False)
                print(f"Saved concatenated {prefix} data to {output_parquet} and {output_parquet.replace('.parquet', '.csv')}")
        else:
            print(f"No {prefix} JSON files found.")
            final_df = None

        return final_df

    #Função para dividir os dados de teste
    def split_test_data(self, df):
        #Separa dados de datahora e latitude/longitude
        test_datahora = df.dropna(subset=['datahora']).drop(columns=['latitude', 'longitude'])
        test_lat_long = df.dropna(subset = ['latitude', 'longitude']).drop(columns=['datahora'])

        #Define caminhos para salvar os dados
        test_datahora_parquet = os.path.join(self.processados_teste_dir, 'test_datahora.parquet')
        test_datahora_csv = os.path.join(self.processados_teste_dir, 'test_datahora.csv')
        test_lat_long_parquet = os.path.join(self.processados_teste_dir, 'test_lat_long.parquet')
        test_lat_long_csv = os.path.join(self.processados_teste_dir, 'test_lat_long.csv')

        #Salva dados em formato Parquet e CSV
        test_datahora.to_parquet(test_datahora_parquet)
        test_datahora.to_csv(test_datahora_csv, index=False)
        test_lat_long.to_parquet(test_lat_long_parquet)
        test_lat_long.to_csv(test_lat_long_csv, index=False)

        print(f"Saved test_datahora to {test_datahora_parquet} and {test_datahora_csv}")
        print(f"Saved test_lat_long to {test_lat_long_parquet} and {test_lat_long_csv}")

    #Função para processar dados recentes iterativamente
    def process_recent_data_iteratively(self, prefix, output_csv):
        all_files = []

        #Caminha pelos diretórios e adiciona arquivos JSON que começam com o prefixo especificado
        for root, _, files in os.walk(self.intermediarios_teste_dir):
            for file in files:
                if file.startswith(prefix) and file.endswith('.json'):
                    all_files.append(os.path.join(root, file))

        final_df_list = []

        #Itera sobre cada arquivo JSON e processa
        for json_file in tqdm(all_files, desc=f"Processing {prefix} files"):
            self.process_and_append_json(json_file, final_df_list, output_csv)

        #Concatena DataFrames se a lista não estiver vazia
        if final_df_list:
            final_df = cudf.concat(final_df_list, ignore_index=True)
        else:
            final_df = None

        return final_df

    #Função para processar e adicionar dados JSON
    def process_and_append_json(self, json_path, df_list, output_csv):
        #Abre e carrega o arquivo JSON
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        #Converte para DataFrame cuDF
        df = cudf.DataFrame.from_pandas(pd.DataFrame(data))

        #Adiciona coluna com a hora extraída do nome do arquivo
        hour_from_file = int(json_path.split('_')[-1].split('.')[0])
        df['hour_from_file'] = hour_from_file

        #Lista de linhas válidas
        linhas_validas = [483, 864, 639, 3, 309, 774, 629, 371, 397, 100, 838, 315, 624, 388, 918, 665, 328, 497, 878, 355, 138, 606, 457, 550, 803, 917, 638, 2336, 399, 298, 867, 553, 565, 422, 756, 186012003, 292, 554, 634, 232, 415, 2803, 324, 852, 557, 759, 343, 779, 905, 108]
        #Filtra linhas válidas
        df = df[df['linha'].astype(str).isin(map(str, linhas_validas))]

        #Converte colunas de data e hora
        df['datahoraenvio'] = cudf.to_datetime(df['datahoraenvio'].astype('int64'), unit='ms')
        df['datahora_converted'] = cudf.to_datetime(df['datahora'].astype('int64'), unit='ms')

        #Converte colunas de latitude e longitude
        df['latitude'] = df['latitude'].str.replace(',', '.').astype('float32')
        df['longitude'] = df['longitude'].str.replace(',', '.').astype('float32')

        #Interpola valores nulos de latitude e longitude
        while df['latitude'].isnull().any() or df['longitude'].isnull().any():
            df['latitude'] = df['latitude'].interpolate().ffill().bfill()
            df['longitude'] = df['longitude'].interpolate().ffill().bfill()

        #Converte a coluna 'velocidade' para int32
        df['velocidade'] = df['velocidade'].astype('int32')
        # Filtra valores de velocidade e coordenadas válidas
        df = df[(df['velocidade'] >= 0) & (df['velocidade'] <= 250)]
        df = df[df['latitude'].between(-90, 90) & df['longitude'].between(-180, 180)]

        #Adiciona DataFrame à lista
        df_list.append(df)

        #Salva intermediário iterativamente em CSV
        if os.path.exists(output_csv):
            df.to_pandas().to_csv(output_csv, mode='a', header=False, index=False)
        else:
            df.to_pandas().to_csv(output_csv, index=False)

    #Função principal para execução do processamento
    def run(self):
        #Obtém arquivos ZIP no diretório base
        zip_files = glob(os.path.join(self.base_dir, '*.zip'))

        if not zip_files:
            raise FileNotFoundError("No zip files found in the specified directory.")

        #Extrai arquivos ZIP
        self.extract_zip_files(zip_files)

        treino_intermediate_parquet = os.path.join(self.processados_teste_dir, 'treino_data.parquet')
        treino_intermediate_csv = os.path.join(self.processados_teste_dir, 'treino_data.csv')

        #Concatena arquivos de treinamento
        treino_data_df = self.process_and_concat_files('2024', treino_intermediate_parquet)

        #Salva dados de treinamento não processados
        if treino_data_df is not None:
            treino_data_df.to_csv(treino_intermediate_csv, index=False)
            print(f"Saved training data to {treino_intermediate_parquet} and {treino_intermediate_csv}")

        #Processa e divide arquivos de teste
        self.process_and_concat_files('teste', os.path.join(self.intermediarios_teste_dir, 'intermediate_test_data.parquet'), is_test=True)


In [ ]:
if __name__ == "__main__":
    base_dir = os.getcwd()
    processor = FinalDataProcessor(base_dir)
    processor.run()

<h3>Pre processing the data</h3>
<p>Agora eu estou aplicando alguns pre processamentos, como a filtragem por linhas ( por alguma razão, a filtragem anterior não funcionou), a diminuição do tamanho da palavra do tipo de dados para tornar o arquivo final menor, e a exclusão de outliers. Outliers são todos os pontos de latitude e longitude que fogem da rota usual da linha ao longo do dia.Outliers incluem garagens e pontos finais. </p>

In [ ]:
linhas_validas = [
    483, 864, 639, 3, 309, 774, 629, 371, 397, 100, 838, 315, 624, 388, 918, 665, 328, 497, 878, 355, 138, 606, 457, 
    550, 803, 917, 638, 2336, 399, 298, 867, 553, 565, 422, 756, 186012003, 292, 554, 634, 232, 415, 2803, 324, 852, 
    557, 759, 343, 779, 905, 108
]

class DataPreprocessing:
    def __init__(self, file_path=None, intermediarios_dir=None, processados_dir=None, output_filename=None, chunk_size=1000000):
        # Inicializa as variáveis de caminho e cria os diretórios necessários
        self.file_path = file_path
        self.intermediarios_dir = intermediarios_dir
        self.processados_dir = processados_dir
        self.cleaned_file = os.path.join(intermediarios_dir, f'cleaned_{output_filename}') if intermediarios_dir and output_filename else None
        self.prepared_file = os.path.join(processados_dir, 'treated', output_filename) if processados_dir and output_filename else None
        self.chunk_size = chunk_size

    def load_data(self):
        return cudf.read_parquet(self.file_path)

    #Função para salvar dados intermediários em formato Parquet
    def save_intermediate_data(self, df, filename):
        df.to_parquet(filename)

    #Função para converter os tipos de dados das colunas para tamanhos menores
    def convert_dtypes(self, df):
        if "latitude" in df.columns and (df["latitude"].dtype == 'object' or df["latitude"].dtype.name == 'category'):
            df["latitude"] = df["latitude"].str.replace(',', '.').astype('float32')
        if "longitude" in df.columns and (df["longitude"].dtype == 'object' or df["longitude"].dtype.name == 'category'):
            df["longitude"] = df["longitude"].str.replace(',', '.').astype('float32')
        if "datahora" in df.columns:
            df['datahora'] = df['datahora'].astype('int64')
        if "linha" in df.columns:
            df['linha'] = df['linha'].astype('int32')
        if "ordem" in df.columns:
            df['ordem'] = df['ordem'].astype('object')
        if "velocidade" in df.columns:
            df['velocidade'] = df['velocidade'].astype('int32')
        if "datahoraenvio" in df.columns:
            df['datahoraenvio'] = df['datahoraenvio'].astype('int64')
        if "datahoraservidor" in df.columns:
            df['datahoraservidor'] = df['datahoraservidor'].astype('int64')
        if "dia_da_semana" in df.columns:
            df['dia_da_semana'] = df['dia_da_semana'].astype('int32')
        if "hora" in df.columns:
            df['hora'] = df['hora'].astype('int16')
        if "diff_timestamp" in df.columns:
            df['diff_timestamp'] = df['diff_timestamp'].astype('float64')
        if "latitude_diff" in df.columns:
            df['latitude_diff'] = df['latitude_diff'].astype('float32')
        if "longitude_diff" in df.columns:
            df['longitude_diff'] = df['longitude_diff'].astype('float32')
        if "distancia" in df.columns:
            df['distancia'] = df['distancia'].astype('float32')
        return df

    #Função para filtrar linhas válidas
    def filter_valid_linhas(self, df):
        if "linha" in df.columns:
            df = df[df['linha'].astype(str).isin(map(str, linhas_validas))]
        return df

    '''
    Isso é mais visível na plotagem de gráficos, mas basicamente, toda linha possui uma 
    rota fixa ao longo do dia. Qualquer rota que desvie dessa rota fixa é considerado
    um outlier. Para encontrar e remover os outliers, a estratégia abaixo está sendo usada.
    '''
    def remove_outliers(self, df, eps=0.001, min_samples=10):
        if os.path.exists(self.cleaned_file):
            df = cudf.read_parquet(self.cleaned_file)
            if 'datahora' in df.columns:
                return df

        if 'datahora_converted' not in df.columns and 'datahora' in df.columns:
            df['datahora_converted'] = cudf.to_datetime(df['datahora'], unit='ms')

        if 'linha' not in df.columns or 'ordem' not in df.columns:
            raise KeyError("Necessary columns are not present in the data.")

        if 'latitude' not in df.columns or 'longitude' not in df.columns:
            return df

        lines = df['linha'].unique().to_pandas()
        cleaned_dfs = []

        #Itera sobre cada linha única e utiliza o DBSCAN para encontrar pontos fora da rota
        for linha in tqdm(lines, desc="Lines", leave=True, dynamic_ncols=True):
            line_data = df[df['linha'] == linha].copy()
            if len(line_data) < min_samples:
                cleaned_dfs.append(line_data)
                continue

            if 'latitude' in line_data.columns and 'longitude' in line_data.columns:
                line_data_pd = line_data[['latitude', 'longitude']].to_pandas()
                db = DBSCAN(eps=eps, min_samples=min_samples)
                labels = db.fit_predict(line_data_pd)

                line_data_pd['labels'] = labels
                non_outliers = line_data_pd[line_data_pd['labels'] != -1]
                non_outliers = non_outliers.copy()

                for col in line_data.columns:
                    if col not in ['latitude', 'longitude', 'labels']:
                        if col in non_outliers:
                            non_outliers[col] = line_data[col].to_pandas().loc[non_outliers.index]

                cleaned_dfs.append(non_outliers)

        if cleaned_dfs:
            cleaned_df = cudf.DataFrame.from_pandas(pd.concat(cleaned_dfs, ignore_index=True))
            self.save_intermediate_data(cleaned_df, self.cleaned_file)
        else:
            raise ValueError("No objects to concatenate")

        return cleaned_df

    #Função para pré-processar os dados
    def preprocess(self, remove_outliers=False):
        #Carrega os dados
        df = self.load_data()
        #Converte os tipos de dados
        df = self.convert_dtypes(df)
        #Filtra linhas válidas
        df = self.filter_valid_linhas(df)
        #Remove outliers se solicitado
        if remove_outliers and 'latitude' in df.columns and 'longitude' in df.columns:
            df = self.remove_outliers(df)
        #Cria diretório para salvar os dados processados
        os.makedirs(os.path.dirname(self.prepared_file), exist_ok=True)
        #Salva os dados processados em formato Parquet e CSV
        df.to_parquet(self.prepared_file)
        df.to_csv(self.prepared_file.replace('.parquet', '.csv'), index=False)
        return df


In [ ]:
if __name__ == "__main__":
    base_dir = os.getcwd()
    base_final_dir = os.path.join(base_dir, 'dados_finais')
    intermediarios_teste_dir = os.path.join(base_final_dir, 'intermediarios_teste')
    processados_teste_dir = os.path.join(base_final_dir, 'processados_teste')

    #Cria o diretório 'treated' dentro do diretório de dados processados de teste
    os.makedirs(os.path.join(processados_teste_dir, 'treated'), exist_ok=True)

    #Inicializa a classe DataPreprocessing para os dados de treino
    treino_preprocessing = DataPreprocessing(
        file_path=os.path.join(processados_teste_dir, 'treino_data.parquet'),
        intermediarios_dir=intermediarios_teste_dir,
        processados_dir=processados_teste_dir,
        output_filename='treated_train_data.parquet'
    )
    #Preprocessa os dados de treino, removendo outliers
    treino_preprocessing.preprocess(remove_outliers=True)

    #Inicializa a classe DataPreprocessing para os dados de hora do teste
    test_datahora_preprocessing = DataPreprocessing(
        file_path=os.path.join(processados_teste_dir, 'test_datahora.parquet'),
        intermediarios_dir=intermediarios_teste_dir,
        processados_dir=processados_teste_dir,
        output_filename='treated_test_datahora.parquet'
    )
    #Preprocessa os dados de hora do teste, sem remover outliers
    test_datahora_preprocessing.preprocess(remove_outliers=False)

    #Inicializa a classe DataPreprocessing para os dados de latitude e longitude do teste
    test_lat_long_preprocessing = DataPreprocessing(
        file_path=os.path.join(processados_teste_dir, 'test_lat_long.parquet'),
        intermediarios_dir=intermediarios_teste_dir,
        processados_dir=processados_teste_dir,
        output_filename='treated_test_lat_long.parquet'
    )
    #Preprocessa os dados de latitude e longitude do teste, sem remover outliers
    test_lat_long_preprocessing.preprocess(remove_outliers=False)


<h3>Processing the data and creating the features</h3>
<p>A estratégia abaixo divide os arquivos de treino em df_datahora e df_lat_long. O objetivo é treinar arquivos separados para cada previsão que precisa ser feita. Além do mais, encoder específico para a coluna ordem foi criada, pois eu preciso transformar essa coluna em número mas sem alterar muito os seus valores. Também foram criadas novas features importantes, especialmente a velocidade média tanto para os datasets de treino quanto para os datasets de teste</p>

In [ ]:
def preprocess_and_split(df):
    #Remove colunas específicas do DataFrame para criar dois datasets de treino distintos
    df_datahora = df.drop(columns=['datahoraservidor', 'datahoraenvio', 'datahora_converted'])
    df_lat_long = df.drop(columns=['datahoraservidor', 'datahoraenvio', 'datahora_converted'])
    return df_datahora, df_lat_long

'''
Aqui estou criando uma função especificamente para transformar a coluna ordem em números.
Isso é importante porque ao rodar alguns modelos de Machine Learning, precisamos passar 
somente valores numéricos. Como eu não queria alterar muito a estrutura dos valores da 
coluna ordem, eu criei uma estrutura própria para cuidar disso. Basicamente todos os 
valores da coluna ordem começa com uma letra seguida de números. Eu peguei essa letra
do início de seus valores e transformei em número seguindo a ordem alfabética.
Então o valor D123456, virou 4123456
'''
def custom_encode_ordem(df, column='ordem'):
    def encode_value(val):
        if isinstance(val, str) and val[0].isalpha():
            letter = val[0].upper()
            number = ord(letter) - ord('A') + 1
            encoded_val = str(number) + val[1:]
            return int(encoded_val)
        elif val.isdigit():
            return int(val)
        return val

    #Aplica a codificação personalizada à coluna 'ordem'
    df[column] = df[column].apply(encode_value).astype('int32')
    return df

#Aqui está a função de feature engineering criando novas colunas
def apply_feature_engineering(df, target):
    try:
        print(f"Converting data types for {target}...")
        conv = DataPreprocessing()
        df = conv.convert_dtypes(df)
    except Exception as e:
        print(f"Error in convert_dtypes: {e}")
        raise

    try:
        print("Criando novas features...")
        if 'datahora' in df.columns:
            #Converte a coluna 'datahora' para datetime
            df['datahora_converted'] = pd.to_datetime(df['datahora'], unit='ms')
        
        if 'datahora_converted' in df.columns and 'linha' in df.columns and 'ordem' in df.columns:
            df = df.sort_values(by=['linha', 'ordem', 'datahora_converted'])
            #Cria novas colunas baseadas em 'datahora_converted'
            df['dia_da_semana'] = df['datahora_converted'].dt.weekday
            df['hora'] = df['datahora_converted'].dt.hour
            df['diff_timestamp'] = df.groupby(['linha', 'ordem'])['datahora_converted'].diff().astype('int64') / 10**6

        if 'latitude' in df.columns and 'longitude' in df.columns:
            df['latitude_diff'] = df.groupby(['linha', 'ordem'])['latitude'].diff().fillna(method='ffill')
            df['longitude_diff'] = df.groupby(['linha', 'ordem'])['longitude'].diff().fillna(method='ffill')

            #Calcula a distância entre pontos geográficos
            lat1 = np.radians(df['latitude'])
            lon1 = np.radians(df['longitude'])
            lat2 = np.radians(df['latitude'] + df['latitude_diff'])
            lon2 = np.radians(df['longitude'] + df['longitude_diff'])

            dlat = lat2 - lat1
            dlon = lon2 - lon1

            a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
            c = 2 * np.arcsin(np.sqrt(a))
            r = 6371
            df['distancia'] = c * r * 1000

        print("Novas features criadas: 'dia_da_semana', 'hora', 'diff_timestamp', 'latitude_diff', 'longitude_diff', 'distancia'")

    except Exception as e:
        print(f"Error in creating new features: {e}")
        raise

    try:
        print(f"Encoding categorical features for {target}...")
        df = custom_encode_ordem(df)
    except Exception as e:
        print(f"Error in encoding categorical features: {e}")
        raise

    return df

'''
Um aspecto muito importante é a velocidade média. Nós temos a coluna velocidade
nos datasets de treino, mas não a velocidade média. Eu me certifiquei de criar essa 
coluna nova. Eu agrupei os dataframes pelas colunas linha e ordem e algumas features
temporais caso a variável alvo fosse datahora e features geoespaciais se as variáveis
alvos fossem latitude e longitude, e a partir disso, calculei a velocidade média.
'''
def add_avg_velocity(train_df, test_df, target):
    conv = DataPreprocessing()
    train_df = conv.convert_dtypes(train_df.copy())
    test_df = conv.convert_dtypes(test_df.copy())

    if target == 'datahora':
        train_df['datahora_converted'] = pd.to_datetime(train_df['datahora'], unit='ms')
        test_df['datahora_converted'] = pd.to_datetime(test_df['datahora'], unit='ms')
        if 'datahora_converted' in train_df.columns:
            train_df['day_of_week'] = train_df['datahora_converted'].dt.weekday
            train_df['hour'] = train_df['datahora_converted'].dt.hour
            train_df['minute'] = train_df['datahora_converted'].dt.minute
            avg_speed = train_df.groupby(['linha', 'ordem', 'day_of_week', 'hour', 'minute']).agg({'velocidade': 'mean'}).reset_index()
            avg_speed.columns = ['linha', 'ordem', 'day_of_week', 'hour', 'minute', 'velocidade_for_datahora']

            train_df = train_df.merge(avg_speed, on=['linha', 'ordem', 'day_of_week', 'hour', 'minute'], how='left')
            train_df['velocidade_for_datahora'] = train_df['velocidade_for_datahora'].ffill().bfill()

            if 'datahora_converted' in test_df.columns:
                test_df['day_of_week'] = test_df['datahora_converted'].dt.weekday
                test_df['hour'] = test_df['datahora_converted'].dt.hour
                test_df['minute'] = test_df['datahora_converted'].dt.minute
                test_df = test_df.merge(avg_speed, on=['linha', 'ordem', 'day_of_week', 'hour', 'minute'], how='left')
                test_df['velocidade_for_datahora'] = test_df['velocidade_for_datahora'].ffill().bfill()
                
                test_df = test_df.rename(columns={'velocidade_for_datahora': 'avg_speed'})
                train_df = train_df.rename(columns={'velocidade_for_datahora': 'avg_speed'})

    elif target == 'lat_long':
        if 'latitude' in train_df.columns and 'longitude' in train_df.columns:
            train_df['lat_temp'] = (train_df['latitude'] * 100).astype(int)
            train_df['lon_temp'] = (train_df['longitude'] * 100).astype(int)

            avg_speed = train_df.groupby(['linha', 'ordem', 'hour_from_file', 'lat_temp', 'lon_temp']).agg({'velocidade': 'mean'}).reset_index()
            avg_speed.columns = ['linha', 'ordem', 'hour_from_file', 'lat_temp', 'lon_temp', 'velocidade_for_lat_long']

            train_df = train_df.merge(avg_speed, on=['linha', 'ordem', 'hour_from_file', 'lat_temp', 'lon_temp'], how='left')
            train_df['velocidade_for_lat_long'] = train_df['velocidade_for_lat_long'].ffill().bfill()

            if 'latitude' in test_df.columns and 'longitude' in test_df.columns:
                test_df['lat_temp'] = (test_df['latitude'] * 100).astype(int)
                test_df['lon_temp'] = (test_df['longitude'] * 100).astype(int)

                test_df = test_df.merge(avg_speed, on=['linha', 'ordem', 'hour_from_file', 'lat_temp', 'lon_temp'], how='left')
                test_df['velocidade_for_lat_long'] = test_df['velocidade_for_lat_long'].ffill().bfill()
                
                test_df = test_df.drop(columns=['lat_temp', 'lon_temp'])
                train_df = train_df.drop(columns=['lat_temp', 'lon_temp'])

                test_df = test_df.rename(columns={'velocidade_for_lat_long': 'avg_speed'})
                train_df = train_df.rename(columns={'velocidade_for_lat_long': 'avg_speed'})

    return train_df, test_df

def sort_columns(df, initial_columns=None, final_columns=None):
    '''
    Ordena as colunas de um DataFrame em ordem alfabética, 
    colocando colunas específicas no início e no final.
    '''
    if initial_columns is None:
        initial_columns = []
    if final_columns is None:
        final_columns = []

    #Obtenha a lista de todas as colunas
    all_columns = list(df.columns)

    #Remova as colunas específicas da lista geral de colunas
    columns_to_sort = [col for col in all_columns if col not in initial_columns + final_columns]

    #Ordene as colunas restantes em ordem alfabética
    sorted_columns = sorted(columns_to_sort)

    #Combine as colunas na ordem desejada
    new_order = initial_columns + sorted_columns + final_columns

    #Reorganiza o DataFrame com as colunas na nova ordem
    df = df[new_order]

    return df

if __name__ == "__main__":
    base_dir = os.getcwd()
    base_final_dir = os.path.join(base_dir, 'dados_finais')
    processados_teste_dir = os.path.join(base_final_dir, 'processados_teste', 'treated')
    feature_modeling_dir = os.path.join(base_final_dir, 'feature_modeling')

    os.makedirs(feature_modeling_dir, exist_ok=True)

    try:
        print("Loading datasets...")
        train_df = pd.read_csv(os.path.join(processados_teste_dir, 'treated_train_data.csv'))
        test_datahora_df = pd.read_csv(os.path.join(processados_teste_dir, 'treated_test_datahora.csv'))
        test_lat_long_df = pd.read_csv(os.path.join(processados_teste_dir, 'treated_test_lat_long.csv'))

        #Divide o conjunto de dados de treino em datahora e lat_long
        print("Splitting training dataset into datahora and lat_long...")
        train_datahora_df, train_lat_long_df = preprocess_and_split(train_df)

        #Adiciona velocidade média aos DataFrames de teste
        print("Adding average velocity to test_datahora dataframe...")
        train_datahora_df, test_datahora_df = add_avg_velocity(train_datahora_df, test_datahora_df, 'datahora')
        
        print("Adding average velocity to test_lat_long dataframe...")
        train_lat_long_df, test_lat_long_df = add_avg_velocity(train_lat_long_df, test_lat_long_df, 'lat_long')

        #Remove a coluna 'velocidade' onde existir. Só queremos a velocidade média
        print("Remove 'velocidade' column where it exists...")
        if 'velocidade' in train_datahora_df.columns:
            train_datahora_df = train_datahora_df.drop(columns=['velocidade'])
        if 'velocidade' in test_datahora_df.columns:
            test_datahora_df = test_datahora_df.drop(columns=['velocidade'])
        if 'velocidade' in train_lat_long_df.columns:
            train_lat_long_df = train_lat_long_df.drop(columns=['velocidade'])
        if 'velocidade' in test_lat_long_df.columns:
            test_lat_long_df = test_lat_long_df.drop(columns=['velocidade'])

        #Aplica feature engineering
        print("Apply feature engineering...")
        train_datahora_df = apply_feature_engineering(train_datahora_df, 'datahora')
        train_lat_long_df = apply_feature_engineering(train_lat_long_df, 'lat_long')
        test_datahora_df = apply_feature_engineering(test_datahora_df, 'datahora')
        test_lat_long_df = apply_feature_engineering(test_lat_long_df, 'lat_long')

        #Remove colunas desnecessárias
        print("Dropping unnecessary columns...")
        train_datahora_df.drop(['latitude_diff', 'longitude_diff', "distancia", "labels", "datahora_converted"], axis=1, inplace=True)
        test_datahora_df.drop(["datahora_converted"], axis=1, inplace=True)
        train_lat_long_df.drop(["labels", "datahora_converted", "dia_da_semana", "hora", "diff_timestamp"], axis=1, inplace=True)

        #Aplica a ordenação das colunas
        print("Applying sorting to the dataframes...")
        train_datahora_df = sort_columns(train_datahora_df, None, final_columns=["latitude", "longitude"])
        test_datahora_df = sort_columns(test_datahora_df, initial_columns = ["id"], final_columns=None)
        train_lat_long_df = sort_columns(train_lat_long_df, None, final_columns=["datahora"])
        test_lat_long_df = sort_columns(test_lat_long_df, initial_columns = ["id"], final_columns=None)

        #Remove linhas com valores nulos
        print("Dropping rows with null values...")
        train_datahora_df = train_datahora_df.dropna()
        train_lat_long_df = train_lat_long_df.dropna()
        test_datahora_df = test_datahora_df.dropna()
        test_lat_long_df = test_lat_long_df.dropna()

        #Salva os conjuntos de dados pré-processados para o treinamento do modelo
        print("Saving preprocessed datasets for later model training...")
        train_datahora_df.to_csv(os.path.join(feature_modeling_dir, 'train_datahora.csv'), index=False)
        train_lat_long_df.to_csv(os.path.join(feature_modeling_dir, 'train_lat_long.csv'), index=False)
        test_datahora_df.to_csv(os.path.join(feature_modeling_dir, 'test_datahora.csv'), index=False)
        test_lat_long_df.to_csv(os.path.join(feature_modeling_dir, 'test_lat_long.csv'), index=False)

        print("Feature engineering and preprocessing complete.")
    except Exception as e:
        print(f"Error in the main processing block: {e}")


<h3>Fazendo Cross Validation</h3>
<p>Aqui irei fazer a validação cruzada. Basicamente, teremos duas previsões a serem feitas de maneira separada. Para isso, irei fazer duas validações cruzadas, testando Random Forest,XGBoost e CatBoost com vários parâmetros. Para cada validação cruzada, o objetivo é encontrar o melhor modelo, com os melhores parâmetros. </p>
<p>No caso da previsão de datahora, a métrica utilizada para a avaliação do melhor modelo é o RMSE. No caso da previsão de latitude, a métrica utilizada para a avaliação do melhor modelo é a média da distância Harversine</p>
<p>Estou usando o Optuna para a validação cruzada e estou usando a aceleração por CUDA. </p>
<p>Após isso, o melhor modelo final é escolhido para cada um das previsões feitas</p>

In [ ]:
#Função auxiliar para cálculo da distância Haversine média, já que não encontrei uma biblioteca que faça esse cálculo diretamente
def mean_haversine_distance(y_true, y_pred):
    #Raio da Terra em quilômetros
    R = 6371.0  
    lat_true, lon_true = cp.radians(y_true[:, 0]), cp.radians(y_true[:, 1])
    lat_pred, lon_pred = cp.radians(y_pred[:, 0]), cp.radians(y_pred[:, 1])
    dlat = lat_pred - lat_true
    dlon = lat_pred - lon_true
    a = cp.sin(dlat / 2) ** 2 + cp.cos(lat_true) * cp.cos(lat_pred) * cp.sin(dlon / 2) ** 2
    c = 2 * cp.arcsin(cp.sqrt(a))
    return cp.mean(R * c)

#Função para executar validação cruzada
def run_cross_validation(df, target_cols, model_types, scoring, objective_func, save_dir, cv_name, is_location=False):
    os.makedirs(save_dir, exist_ok=True)
    best_params = {}

    #Itera sobre cada tipo de modelo especificado
    for model_type in model_types:
        print(f"Running Optuna for {model_type} on {cv_name} prediction...")
        study_path = os.path.join(save_dir, f"{cv_name}_{model_type}_study.pkl")

        #Carrega estudo existente ou cria um novo. Isso é pra evitar refazer o que foi feito se der algum erro
        if os.path.exists(study_path):
            print(f"Loading existing study for {model_type} on {cv_name} prediction...")
            study = joblib.load(study_path)
        else:
            study = optuna.create_study(direction='minimize')
            study.optimize(lambda trial: objective_func(trial, df, target_cols, is_location, model_type, scoring, cv_name), n_trials=10)
            joblib.dump(study, study_path)

        #Armazena os melhores parâmetros encontrados
        if study.best_trial:
            best_params[model_type] = study.best_trial.params
        else:
            print(f"No completed trials for {model_type} on {cv_name} prediction. Skipping...")
            best_params[model_type] = None

    return best_params

'''
Função objetivo para otimização. Essa função define os modelos e parâmetros que serão
testados usando Optuna
'''
def objective(trial, df, target_cols, is_location, model_type, scoring, cv_name):
    try:
        #Define os parâmetros para cada tipo de modelo
        if model_type == 'xgboost':
            params = {
                'verbosity': 0,
                'objective': 'reg:squarederror',
                'booster': 'gbtree',
                'tree_method': 'gpu_hist',
                'gpu_id': 0,
                'eta': trial.suggest_float('eta', 1e-4, 1e-1, log=True),
                'max_depth': trial.suggest_int('max_depth', 2, 20),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'enable_categorical': True
            }
            model = xgb.XGBRegressor(**params)
        elif model_type == 'cuml_rf':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, 20),
                'max_features': trial.suggest_float('max_features', 0.5, 1.0)
            }
            if is_location:
                model_lat = cuRF(**params)
                model_lon = cuRF(**params)
            else:
                model = cuRF(**params)
        elif model_type == 'catboost':
            params = {
                'iterations': trial.suggest_int('iterations', 100, 1000),
                'depth': trial.suggest_int('depth', 4, 10),
                'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 1e-1, log=True),
                'task_type': 'GPU',
                'devices': '0'
            }
            model = CatBoostRegressor(**params)
        else:
            raise ValueError(f"Unsupported model type: {model_type}")

        '''
        Define a validação cruzada. Se for para previsão de Latitude e Longitude
        precisamos utilizar essa estratégia de MultiOutputRegressor, já que se trata
        da previsão de duas colunas
        '''
        if is_location and model_type not in ['cuml_rf']:
            model = MultiOutputRegressor(model)

        scores = []
        tss = TimeSeriesSplit(n_splits=3)

        X = df.drop(columns=target_cols)
        y = df[target_cols]

        prediction_dir = os.path.join("dados_finais", "cross_validation", "intermediate_predictions", model_type)
        os.makedirs(prediction_dir, exist_ok=True)

        #Itera sobre cada divisão da validação cruzada
        for train_index, test_index in tqdm(tss.split(X), desc="Cross-validation", leave=False):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            #Verifica e trata valores NaN
            if X_train.isna().sum().sum() > 0 or y_train.isna().sum().sum() > 0 or X_test.isna().sum().sum() > 0 or y_test.isna().sum().sum() > 0:
                raise ValueError(f"NaN values found in the data at fold {train_index}-{test_index}")

            '''
            Essa parte abaixo é a estratégia que consiste em treinar as duas horas 
            consecutivas anteriores, para prever a hora seguinte. Essa estratégia
            foi sugerida pelo professor Zimbrão. A coluna utilizada como referência
            para a hora foi a hour_from_file. Essa coluna foi criada logo no começo
            do processamento dos dados e os seus valores foram retirados dos próprios 
            nomes dos arquivos json, que continham a hora nos 2 últimos caracteres.
            '''
            unique_hours = sorted(X_test['hour_from_file'].to_pandas().unique())

            '''
            Itera sobre cada hora única. Como o kernel estava quebrando toda vez
            que eu precisei adotar essa estratégia de treinar as duas horas anteriores
            para prever as horas seguintes, eu resolvi fazer uma previsão de 1 hora
            por vez, salvando essas previsões em arquivos intermediários. No final,
            iremos usar todas essas previsões para criar um modelo preditivo só para 
            cada previsão que precisa ser feita: datahora ou latitude e longitude.
            '''
            for i in range(2, len(unique_hours)):
                train_hours = unique_hours[i-2:i]
                test_hour = unique_hours[i]

                preds_path = os.path.join(prediction_dir, f"{cv_name}_{model_type}_preds_{test_hour}.npy")
                actuals_path = os.path.join(prediction_dir, f"{cv_name}_{model_type}_actuals_{test_hour}.npy")

                #Verifica se os arquivos intermediários já existem
                if os.path.exists(preds_path) and os.path.exists(actuals_path):
                    print(f"Intermediate files for hour {test_hour} already exist. Skipping...")
                    continue

                X_train_hour = X_train[X_train['hour_from_file'].isin(train_hours)]
                y_train_hour = y_train.loc[X_train_hour.index]

                X_test_hour = X_test[X_test['hour_from_file'] == test_hour]
                y_test_hour = y_test.loc[X_test_hour.index]

                #Verifica se o conjunto de treino ou teste está vazio
                if X_train_hour.empty or X_test_hour.empty:
                    print(f"Skipping empty train or test set for hour {test_hour}")
                    continue

                '''
                Treina e faz as previsões hora a hora. No final, apenas concateno
                todas essas previsões. Essa estratégia de fazer as previsões de 
                forma iterativa foi necessário apenas para alguns modelos.
                '''
                if is_location and model_type == 'cuml_rf':
                    model_lat.fit(X_train_hour, y_train_hour['latitude'])
                    model_lon.fit(X_train_hour, y_train_hour['longitude'])
                    preds_lat = model_lat.predict(X_test_hour)
                    preds_lon = model_lon.predict(X_test_hour)
                    preds = cp.vstack((preds_lat, preds_lon)).T
                else:
                    model.fit(X_train_hour.to_pandas(), y_train_hour.to_pandas())
                    preds = model.predict(X_test_hour.to_pandas())

                '''
                Como eu precisei utilizar recentemente o cudf para a aceleração por
                GPU e Cuda, em alguns pontos do código, eu precisei retornar os 
                objetos para Pandas e/ou numpy.
                '''
                #Converte previsões para numpy array se necessário
                if isinstance(preds, (pd.Series, pd.DataFrame)):
                    preds = preds.to_numpy()
                elif isinstance(preds, cp.ndarray):
                    preds = preds.get()

                # Garante que as previsões tenham o mesmo formato que os valores reais
                if len(target_cols) == 1:
                    preds = preds.reshape(-1, 1)
                else:
                    preds = preds.reshape(-1, len(target_cols))

                cp.save(preds_path, preds)
                cp.save(actuals_path, y_test_hour.to_pandas().values)

                if preds.shape != y_test_hour.shape:
                    print(f"Inconsistent shape detected in predictions for hour {test_hour}. Skipping...")
                    continue

        #Agrega todas as previsões e valores reais
        predictions = []
        actuals = []

        #Itera sobre cada hora única novamente para agregar previsões e valores reais
        for i in range(2, len(unique_hours)):
            test_hour = unique_hours[i]

            preds_path = os.path.join(prediction_dir, f"{cv_name}_{model_type}_preds_{test_hour}.npy")
            actuals_path = os.path.join(prediction_dir, f"{cv_name}_{model_type}_actuals_{test_hour}.npy")

            if os.path.exists(preds_path) and os.path.exists(actuals_path):
                preds = cp.load(preds_path)
                actuals_data = cp.load(actuals_path)

                #Garante que previsões e valores reais tenham o mesmo formato
                #É uma estratégia para que possamos concatenar as previsões sem conflitos.
                preds = preds.reshape(-1, actuals_data.shape[1])

                if preds.shape[0] != actuals_data.shape[0]:
                    print(f"Inconsistent sample size between actuals ({actuals_data.shape[0]}) and predictions ({preds.shape[0]}). Skipping hour {test_hour}...")
                    continue

                predictions.append(preds)
                actuals.append(actuals_data)

        #Verifica se há previsões e valores reais válidos
        if predictions and actuals:
            predictions = cp.vstack(predictions)
            actuals = cp.vstack(actuals)
        else:
            raise ValueError("No valid predictions were aggregated.")

        #Calcula a métrica de avaliação: ou RMSE para prever datahora ou 
        #Mean Haversine Distance para latitude e longitude.
        if scoring == 'rmse':
            score = mean_squared_error(actuals.get(), predictions.get(), squared=False)
        elif scoring == 'mean_haversine_distance':
            if is_location:
                if predictions.shape[1] == 2 and actuals.shape[1] == 2:
                    score = mean_haversine_distance(actuals, predictions)
                else:
                    raise ValueError("Prediction and actual shapes do not match the expected latitude/longitude format")
            else:
                raise ValueError("mean_haversine_distance is not applicable for datahora prediction")

        scores.append(score)

        if len(scores) == 0:
            raise ValueError("No scores were computed. Check data and model configurations.")
        return cp.mean(cp.array(scores))

    except Exception as e:
        print(f"Error in trial: {e}")
        raise ValueError("No scores were computed. Check data and model configurations.")

#Função para criar o modelo final a partir dos melhores parâmetros e melhores modelos 
#avaliados pela validação cruzada
def create_final_model(X, y, best_model_name, best_params, save_dir, model_type):
    final_model_path = os.path.join(save_dir, f'final_model_{model_type}.pkl')

    if os.path.exists(final_model_path):
        print(f"Model file {final_model_path} already exists. Skipping model creation.")
        return joblib.load(final_model_path)

    #Define o modelo com os melhores parâmetros
    if best_model_name == 'xgboost':
        model = xgb.XGBRegressor(**best_params)
    elif best_model_name == 'cuml_rf':
        model_lat = cuRF(**best_params)
        model_lon = cuRF(**best_params)
    elif best_model_name == 'catboost':
        model = CatBoostRegressor(**best_params)
    else:
        raise ValueError(f"Unsupported model type: {best_model_name}")

    #Treina e salva o modelo
    if model_type == 'lat_long' and best_model_name == 'cuml_rf':
        model_lat.fit(X.to_pandas(), y['latitude'].to_pandas())
        model_lon.fit(X.to_pandas(), y['longitude'].to_pandas())
        joblib.dump((model_lat, model_lon), final_model_path)
    else:
        model.fit(X.to_pandas(), y.to_pandas())
        joblib.dump(model, final_model_path)

    return model

def main():
    base_dir = os.getcwd()
    feature_modeling_dir = os.path.join(base_dir, 'dados_finais', 'feature_modeling')
    cross_validation_dir = os.path.join(base_dir, 'dados_finais', 'cross_validation')
    final_model_dir = os.path.join(cross_validation_dir, 'final_models')
    os.makedirs(final_model_dir, exist_ok=True)

    try:
        #Carrega os conjuntos de dados
        train_lat_long = cudf.read_csv(os.path.join(feature_modeling_dir, 'train_lat_long.csv'))
        train_datahora = cudf.read_csv(os.path.join(feature_modeling_dir, 'train_datahora.csv'))

        model_types = ['xgboost', 'cuml_rf', 'catboost']

        #Executa a validação cruzada para a previsão de datahora
        print("Running cross-validation for datahora prediction...")
        best_params_datahora = run_cross_validation(train_lat_long, ['datahora'], model_types, 'rmse', objective, cross_validation_dir, 'datahora')

        #Executa a validação cruzada para a previsão de latitude e longitude
        print("Running cross-validation for lat_long prediction...")
        best_params_lat_long = run_cross_validation(train_datahora, ['latitude', 'longitude'], model_types, 'mean_haversine_distance', objective, cross_validation_dir, 'lat_long', is_location=True)

        #Cria os modelos finais para a previsão de datahora
        print("Creating final models for datahora prediction...")
        for model_type in model_types:
            if best_params_datahora[model_type] is not None:
                X_datahora = train_lat_long.drop(columns=['datahora'])
                y_datahora = train_lat_long['datahora']
                create_final_model(X_datahora, y_datahora, model_type, best_params_datahora[model_type], final_model_dir, 'datahora')

        #Cria os modelos finais para a previsão de latitude e longitude
        print("Creating final models for lat_long prediction...")
        for model_type in model_types:
            if best_params_lat_long[model_type] is not None:
                X_lat_long = train_datahora.drop(columns=['latitude', 'longitude'])
                y_lat_long = train_datahora[['latitude', 'longitude']]
                create_final_model(X_lat_long, y_lat_long, model_type, best_params_lat_long[model_type], final_model_dir, 'lat_long')

    except Exception as e:
        print(f"Error in main: {e}")

if __name__ == "__main__":
    main()


<h1>Realizando as previsões</h1>
<p>Como não ficou claro como as previsões deveriam ser feitas eu tive que assumir algumas coisas:
<li>Todos os dados usados foram coletados exclusivamente do último post em relação à tarefa 3 no Moodle</li>
<li>Os dados foram treinados exclusivamente usando os dados criados a partir dos arquivos .JSON cujo nome começa com 2024</li>
<li>Para fazer a avaliação do modelo durante a validação cruzada, os datasets de treino foram divididos entre treino e test, como normalmente é feito</li>
<li>Os modelos criados durante a fase de treinamento foram utilizados nos datasets de teste disponibilizados no último link.</li>
<li>Como previamente os datasets de treino haviam sido divididos para aplicar os modelos de previsão de datahora e para aplicar os modelos de previsão de latitude e longitude, colunas específicas foram adicionadas ou retiradas aos dois datasets de teste</li>
<li>Para o dataset de teste onde o modelo de prever a datahora foi aplicado, as colunas temporais foram retiradas para evitar o data leakage</li>
<li>Para o dataset de teste onde o modelo de prever latitude e longitude foi aplicado, as colunas com informações geoespaciais foram retiradas para evitar o data leakage.</li>
<li>Após a previsão foram criados dois arquivos: um contendo todas as colunas anteriores, mais as previsões de datahora chamado predicted_datahora e outro contendo todas as colunas anteriores mais as previsões de latitude e longitude chamado predicted_lat_long</li>
 </p>

In [ ]:
def make_predictions(model_path, test_data_path, target_cols, prediction_file_name, required_features, output_dir):
    #Carrega o modelo
    model = joblib.load(model_path)

    #Carrega os dados de teste
    test_data = pd.read_csv(test_data_path)

    #Salva a coluna 'id' e a remove dos dados de teste
    id_column = test_data['id']
    test_data = test_data.drop(columns=['id'], errors='ignore')

    #Verifica se todas as features necessárias estão presentes
    missing_features = set(required_features) - set(test_data.columns)
    if missing_features:
        raise ValueError(f"Missing required features in test data: {missing_features}")

    #Garante que as colunas estejam na ordem correta
    test_data = test_data[required_features]

    #Faz as previsões
    predictions = model.predict(test_data)

    #Garante que as previsões tenham o formato correto para modelos MultiOutput
    if isinstance(predictions, (pd.Series, pd.DataFrame)):
        predictions = predictions.to_numpy()
    elif isinstance(predictions, cp.ndarray):
        predictions = predictions.get()

    if predictions.ndim == 1:
        predictions = predictions.reshape(-1, 1)

    #Adiciona as previsões aos dados de teste
    predictions_df = pd.DataFrame(predictions, columns=target_cols)
    test_data_with_predictions = pd.concat([test_data, predictions_df], axis=1)

    #Adiciona a coluna 'id' de volta aos dados de teste
    test_data_with_predictions['id'] = id_column

    #Salva os dados de teste modificados com as previsões em um arquivo CSV
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{prediction_file_name}.csv")
    test_data_with_predictions.to_csv(output_path, index=False)
    print(f"Predictions added and saved to {output_path}")

def execute_predictions():
    base_dir = os.getcwd()
    cross_validation_dir = os.path.join(base_dir, 'dados_finais', 'cross_validation', 'final_models')
    feature_modeling_dir = os.path.join(base_dir, 'dados_finais', 'feature_modeling')
    output_dir = os.path.join(base_dir, 'dados_finais', 'respostas')

    #Caminhos para os modelos finais
    datahora_model_path = os.path.join(cross_validation_dir, 'final_model_datahora.pkl')
    lat_long_model_path = os.path.join(cross_validation_dir, 'final_model_lat_long.pkl')

    #Caminhos para os dados de teste
    test_lat_long_path = os.path.join(feature_modeling_dir, 'test_lat_long.csv')
    test_datahora_path = os.path.join(feature_modeling_dir, 'test_datahora.csv')

    #Features necessárias
    datahora_required_features = ['avg_speed', 'distancia', 'hour_from_file', 'latitude', 'latitude_diff', 'linha', 'longitude', 'longitude_diff', 'ordem']
    lat_long_required_features = ['avg_speed', 'datahora', 'day_of_week', 'dia_da_semana', 'diff_timestamp', 'hora', 'hour', 'hour_from_file', 'linha', 'minute', 'ordem']

    #Faz previsões para latitude e longitude usando final_model_lat_long.pkl com test_datahora.csv
    print("Predicting latitude and longitude...")
    make_predictions(lat_long_model_path, test_datahora_path, ['latitude', 'longitude'], 'predicted_lat_long', lat_long_required_features, output_dir)

    #Faz previsões para datahora usando final_model_datahora.pkl com test_lat_long.csv
    print("Predicting datahora...")
    make_predictions(datahora_model_path, test_lat_long_path, ['datahora'], 'predicted_datahora', datahora_required_features, output_dir)

#Executa as previsões
execute_predictions()


<h1>Criando os arquivos de submissão</h1>
<p>Como especificado no enunciado, o formato dos arquivos de submissão deveriam ser em .json. Então eu fiz um arquivo .json para cada previsão feita. <b>O grande problema com esse approach é que não será possível submeter essas respostas, haja vista que isso gerou arquivos .json muito grande. </b> </p>




In [ ]:
#Para a previsão de datahora
#O arquivo .json foi criado no seguinte formato:

'''
{ 
   "aluno": "Your Name",
   "datahora": "2024-05-20 13:00:00" #Valor aleatório que não foi explicado no enunciado,
   "previsoes": [ 
                  [400172783234, 1716205511000] #Id e a previsão de datahora previstos, 
                  [282474448123, 1716204264000]
                ], 
   "senha": "your_password"
}
'''

In [ ]:
#Para a previsão de latitude e longitude
#O arquivo .json foi criado no seguinte formato:
'''
{ 
   "aluno": "Your Name",
   "datahora": "2024-05-20 13:00:00" #Valor aleatório que não foi explicado no enunciado,
   "previsoes": [ 
                  [362511850614, -22.82553, -43.16925] #Id e valores de latitude e longitude previstos, 
                  [288961216441, -23.0202, -43.46159]
                ], 
   "senha": "your_password"
}
'''

In [ ]:
'''
def create_submission_json(predictions_file, aluno, senha, prediction_type, output_dir):
    #Carrega o arquivo de previsões
    predictions = pd.read_csv(predictions_file)
    
    #Garante que a coluna 'datahora' esteja corretamente formatada
    if 'datahora' in predictions.columns:
        predictions['datahora'] = pd.to_datetime(predictions['datahora'], unit='ms')
        
    #Cria a coluna 'submission_datahora' combinando data e hora, com minutos e segundos
    if 'datahora' in predictions.columns and 'hour_from_file' in predictions.columns:
        predictions['submission_datahora'] = predictions['datahora'].dt.strftime('%Y-%m-%d') + ' ' + predictions['hour_from_file'].astype(str).str.zfill(2) + ':00:00'
    
    #Garante que as colunas 'latitude' e 'longitude' estejam corretamente formatadas
    if 'latitude' in predictions.columns and 'longitude' in predictions.columns:
        predictions['latitude'] = predictions['latitude'].astype(float)
        predictions['longitude'] = predictions['longitude'].astype(float)
    
    #Cria diretório de saída, se não existir
    os.makedirs(output_dir, exist_ok=True)
    
    #Processa cada grupo de previsões por 'submission_datahora'
    for submission_datahora, group in predictions.groupby('submission_datahora'):
        initial_datahora = submission_datahora
        if prediction_type == 'datahora':
            #Cria a lista de previsões para 'datahora'
            previsoes = [[int(row['id']), row['datahora'].timestamp() * 1000] for _, row in group.iterrows()]
        elif prediction_type == 'lat_long':
            #Cria a lista de previsões para 'lat_long'
            previsoes = [[int(row['id']), row['latitude'], row['longitude']] for _, row in group.iterrows()]
        else:
            raise ValueError("Invalid prediction type. Must be 'datahora' or 'lat_long'.")
        
        #Constrói o objeto JSON
        json_object = {
            "aluno": aluno,
            "datahora": initial_datahora,
            "previsoes": previsoes,
            "senha": senha
        }
        
        #Salva o objeto JSON em um arquivo
        formatted_date = initial_datahora.replace(' ', '_').replace(':', '_')
        output_file = os.path.join(output_dir, f"resposta_{prediction_type}_{formatted_date}.json")
        with open(output_file, "w") as json_file:
            json.dump(json_object, json_file, indent=4)
        print(f"Submission file saved to {output_file}")

#Define as credenciais do aluno
aluno = "Wagner Luiz Lobo Ferreira"
senha = "muramassa_coi"
base_dir = os.getcwd()
resposta_dir = os.path.join(base_dir, 'dados_finais', 'respostas')

#Cria o arquivo JSON de submissão para previsões de 'datahora'
create_submission_json(
    predictions_file=os.path.join(resposta_dir, 'predicted_datahora.csv'),
    aluno=aluno,
    senha=senha,
    prediction_type='datahora',
    output_dir=os.path.join(resposta_dir, 'submission')
)

#Cria o arquivo JSON de submissão para previsões de 'latitude' e 'longitude'
create_submission_json(
    predictions_file=os.path.join(resposta_dir, 'predicted_lat_long.csv'),
    aluno=aluno,
    senha=senha,
    prediction_type='lat_long',
    output_dir=os.path.join(resposta_dir, 'submission')
)
'''

In [ ]:
def create_submission_json(predictions_file, prediction_type, output_dir):
    #Carrega o arquivo de previsões
    predictions = pd.read_csv(predictions_file)
    
    #Garante que a coluna 'datahora' esteja corretamente formatada
    if 'datahora' in predictions.columns:
        predictions['datahora'] = pd.to_datetime(predictions['datahora'], unit='ms')
        
    #Cria a coluna 'submission_datahora' combinando data e hora, com minutos e segundos
    if 'datahora' in predictions.columns and 'hour_from_file' in predictions.columns:
        predictions['submission_datahora'] = predictions['datahora'].dt.strftime('%Y-%m-%d') + ' ' + predictions['hour_from_file'].astype(str).str.zfill(2) + ':00:00'
    
    #Garante que as colunas 'latitude' e 'longitude' estejam corretamente formatadas
    if 'latitude' in predictions.columns and 'longitude' in predictions.columns:
        predictions['latitude'] = predictions['latitude'].astype(float)
        predictions['longitude'] = predictions['longitude'].astype(float)
    
    #Cria diretório de saída, se não existir
    os.makedirs(output_dir, exist_ok=True)
    
    #Processa cada grupo de previsões por 'submission_datahora'
    for submission_datahora, group in predictions.groupby('submission_datahora'):
        initial_datahora = submission_datahora
        if prediction_type == 'datahora':
            #Cria a lista de previsões para 'datahora'
            previsoes = [[int(row['id']), row['datahora'].timestamp() * 1000] for _, row in group.iterrows()]
        elif prediction_type == 'lat_long':
            #Cria a lista de previsões para 'lat_long'
            previsoes = [[int(row['id']), row['latitude'], row['longitude']] for _, row in group.iterrows()]
        else:
            raise ValueError("Invalid prediction type. Must be 'datahora' or 'lat_long'.")
        
        #Constrói o objeto JSON
        json_object = {
            #"aluno": aluno,
            "datahora": initial_datahora,
            "previsoes": previsoes
            #"senha": senha
        }
        
        #Salva o objeto JSON em um arquivo
        formatted_date = initial_datahora.replace(' ', '_').replace(':', '_')
        output_file = os.path.join(output_dir, f"resposta_{prediction_type}_{formatted_date}.json")
        with open(output_file, "w") as json_file:
            json.dump(json_object, json_file, indent=4)
        print(f"Submission file saved to {output_file}")

#Define as credenciais do aluno
#aluno = "Wagner Luiz Lobo Ferreira"
#senha = "muramassa_coi"
base_dir = os.getcwd()
resposta_dir = os.path.join(base_dir, 'dados_finais', 'respostas')

#Cria o arquivo JSON de submissão para previsões de 'datahora'
create_submission_json(
    predictions_file=os.path.join(resposta_dir, 'predicted_datahora.csv'),
    #aluno=aluno,
    #senha=senha,
    prediction_type='datahora',
    output_dir=os.path.join(resposta_dir, 'submission')
)

#Cria o arquivo JSON de submissão para previsões de 'latitude' e 'longitude'
create_submission_json(
    predictions_file=os.path.join(resposta_dir, 'predicted_lat_long.csv'),
    #aluno=aluno,
    #senha=senha,
    prediction_type='lat_long',
    output_dir=os.path.join(resposta_dir, 'submission')
)


In [1]:
import os
import requests
import json
import time

# Define the directory containing the JSON files
base_dir = os.getcwd()
resposta_dir = os.path.join(base_dir, 'dados_finais', 'respostas', 'submission')

# Define the API endpoint
api_url = 'https://barra.cos.ufrj.br:443/rest/rpc/avalia'

# Define the student credentials
aluno = "Wagner Luiz Lobo Ferreira"
senha = "muramassa_coi"

def send_json_file(file_path):
    with open(file_path, 'r') as json_file:
        json_data = json.load(json_file)
    
    # Add the student credentials to the JSON data
    json_data['aluno'] = aluno
    json_data['senha'] = senha
    
    while True:
        try:
            response = requests.post(
                api_url,
                headers={
                    'accept': 'application/json',
                    'Content-Type': 'application/json'
                },
                data=json.dumps(json_data),
                timeout=10  # Set a timeout for the request
            )
            if response.status_code == 200:
                print(f"File {os.path.basename(file_path)} sent successfully.")
                break
            else:
                print(f"Failed to send file {os.path.basename(file_path)}. Status code: {response.status_code}, Response: {response.text}")
                break
        except requests.exceptions.RequestException as e:
            print(f"Error sending file {os.path.basename(file_path)}: {e}. Retrying in 5 seconds...")
            time.sleep(5)  # Wait for 5 seconds before retrying

# Iterate over all JSON files in the directory and send each one
for file_name in os.listdir(resposta_dir):
    if file_name.endswith('.json'):
        file_path = os.path.join(resposta_dir, file_name)
        send_json_file(file_path)


File resposta_datahora_2024-05-17_07_00_00.json sent successfully.
Failed to send file resposta_datahora_2024-05-17_11_00_00.json. Status code: 400, Response: {"code":"P0001","details":"Encontrado: [[number, number]]","hint":"Esperado: [number,number,number] (ex: [ [362511850614, -22.82553, -43.16925 ], [288961216441, -23.0202, -43.46159 ],  ... ] )","message":"O parâmetro 'previsoes' deve ser um array contendo arrays de previsões para latitude,longitude"}
Failed to send file resposta_datahora_2024-05-17_13_00_00.json. Status code: 400, Response: {"code":"P0001","details":"Exemplo: teste-2024-05-20_13.json ==> 2024-05-20 13:00:00","hint":"O testes válidos estão nos arquivos teste-data_hora.json","message":"Teste não encontrado: 2024-05-17 13:00:00"}
Failed to send file resposta_datahora_2024-05-17_19_00_00.json. Status code: 400, Response: {"code":"P0001","details":"Encontrado: [[number, number]]","hint":"Esperado: [number,number,number] (ex: [ [362511850614, -22.82553, -43.16925 ], [2